In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
train_data = "./data/train.csv"
test_data = "./data/test.csv"

In [3]:
train_df = pd.read_csv(train_data)

In [4]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
dup_train_df = train_df[train_df.is_duplicate == 1]

In [6]:
dup_edges = pd.DataFrame({
    'source': dup_train_df.qid1.values.tolist(),
    'target': dup_train_df.qid2.values.tolist(),
    'weight': dup_train_df.is_duplicate.values.tolist(),
})

dup_train_G = nx.from_pandas_edgelist(dup_edges, edge_attr=True)

In [7]:
train_class_graph_list = list(nx.connected_component_subgraphs(dup_train_G))

In [8]:
len(train_class_graph_list)

60460

In [14]:
list(train_class_graph_list[0])

[11, 12]

In [21]:
dup_train_df[dup_train_df.qid1 == 11].question1.values[0]

'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?'

In [30]:
%%time
stacked_clusters_train_df = pd.DataFrame(columns=['q_class','qid','question'])

for c, G in enumerate(train_class_graph_list):
    q_class = c
    for qid in list(G):
        if not dup_train_df[dup_train_df.qid1 == qid].empty:
            q_question = dup_train_df[dup_train_df.qid1 == qid].question1.values[0]
        elif not dup_train_df[dup_train_df.qid2 == qid].empty:
            q_question = dup_train_df[dup_train_df.qid2 == qid].question2.values[0]
        else:
            print("Question for qid {} not found. ".format(qid))
        if q_question:
            stacked_clusters_train_df = stacked_clusters_train_df.append(
                pd.DataFrame([[q_class, qid, q_question]], columns=['q_class','qid','question'])
            )

CPU times: user 49min 21s, sys: 2min 24s, total: 51min 45s
Wall time: 52min 13s


In [31]:
stacked_clusters_train_df.to_csv("./data/stacked_clusters_train_df.csv", index=False)

In [33]:
stacked_clusters_train_df.reset_index(inplace=True)

In [34]:
stacked_clusters_train_df.head()

,index,q_class,qid,question
0,0,0,11,Astrology: I am a Capricorn Sun Cap moon and c...
1,0,0,12,"I'm a triple Capricorn (Sun, Moon and ascendan..."
2,0,1,16,What should I do to be a great geologist?
3,0,1,15,How can I be a good geologist?
4,0,2,24,How can I see all my Youtube comments?
